# LFT 12 — Gravity from Constraint Geometry

## Objective
Develop a first toy model of **gravity as curvature of the logical geometry** induced by **constraint density**, consistent with the Logic Field Theory (LFT) program:
- **Matter/Energy** ≙ **dense, persistent logical constraints** in the information space.
- **Gravity** ≙ **distortion of L-flow** and **geodesics** in the emergent state geometry (permutohedral proxy), caused by constraint clusters.

This notebook provides:
1. A discrete **state-space proxy** (2D lattice) for local permutohedral geometry.
2. A **constraint potential** that slows L-flow (time dilation) and warps shortest paths (geodesic bending).
3. A **redshift demo** (two clocks at different constraint densities).
4. A **geodesic demo** (path bending around a “mass” of constraints).

> **Placement in sequence.** Extends LFT_06 (3+1 spacetime) by letting constraint density shape dynamics; complements 10 (Observer) where constraints drive collapse; aligns with 07 (Born) and 11 (Tsirelson) that use the same global Gram/feasibility logic.

## 1) Discrete State-Space Proxy
We use a 2D lattice as a **local chart** of the emergent geometry. Each node is a micro-state; edges represent admissible adjacent transitions of the L-flow.

A **constraint mass** centered at \((x_0,y_0)\) induces a scalar potential \(\Phi\), higher near the mass and decaying with distance. L-flow “speed” is reduced by a factor depending on \(\Phi\).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

nx, ny = 61, 61
x = np.linspace(-3, 3, nx)
y = np.linspace(-3, 3, ny)
X, Y = np.meshgrid(x, y)

# Constraint “mass” parameters
x0, y0 = 0.0, 0.0
alpha = 3.5  # coupling strength of constraints to flow slowdown
r2 = (X - x0)**2 + (Y - y0)**2
Phi = 1.0 / (1.0 + r2)  # simple decaying potential (bounded, smooth)

# L-flow local time-step multiplier: dt_local = 1 + alpha * Phi
dt_local = 1.0 + alpha * Phi

plt.figure(figsize=(6,5))
plt.imshow(Phi, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
plt.colorbar(label='Constraint potential $\Phi$')
plt.title('Constraint density (proxy potential $\Phi$)')
plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_Phi_field.png', dpi=160)
plt.show()

Phi.min(), Phi.max()

## 2) Time Dilation (Redshift) from L-flow Slowdown
A **local clock** advances by one tick per **global step** scaled by the local slowdown factor. If the L-flow is slower near high \(\Phi\), then clocks near the constraint mass tick **fewer** times over the same global step budget — an analogue of **gravitational redshift**.

In [2]:
def simulate_clock_ticks(Phi, alpha=3.5, steps=5000, pos=(0,0)):
    i, j = pos
    dt = 1.0 + alpha * Phi[j, i]
    # Each global step contributes 1/dt local ticks
    return steps / dt

# Select two positions: near mass and far away
near = (nx//2, ny//2)              # center
far = (nx//2, ny//2 + 20)          # displaced along y

ticks_near = simulate_clock_ticks(Phi, alpha=alpha, steps=10000, pos=near)
ticks_far  = simulate_clock_ticks(Phi, alpha=alpha, steps=10000, pos=far)
redshift = ticks_far / ticks_near
ticks_near, ticks_far, redshift

In [3]:
# Visualize ticks per location across the grid to show redshift profile
Ticks = 10000 / (1.0 + alpha * Phi)
plt.figure(figsize=(6,5))
plt.imshow(Ticks, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
plt.colorbar(label='ticks over fixed global steps')
plt.title('Time dilation: slower L-flow near constraint mass')
plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_Redshift_map.png', dpi=160)
plt.show()

## 3) Geodesic Bending via Weighted Paths
Define edge weights as the **local traversal cost** proportional to the slowdown factor, i.e., \(w = 1 + \alpha\,\Phi\). Shortest paths (Dijkstra) then **avoid** high-\(\Phi\) regions, bending around the constraint mass — a discrete analogue of **geodesics in curved spacetime**.

In [4]:
import heapq

W = 1.0 + alpha * Phi  # traversal cost field

def dijkstra_path(W, start, goal):
    ny, nx = W.shape
    INF = 1e18
    dist = np.full((ny,nx), INF)
    prev = np.full((ny,nx,2), -1, dtype=int)
    sx, sy = start; gx, gy = goal
    dist[sy, sx] = 0.0
    pq = [(0.0, sx, sy)]
    while pq:
        d, x0, y0 = heapq.heappop(pq)
        if d>dist[y0,x0]:
            continue
        if (x0,y0)==(gx,gy):
            break
        for dx,dy in [(1,0),(-1,0),(0,1),(0,-1)]:
            x1, y1 = x0+dx, y0+dy
            if 0<=x1<nx and 0<=y1<ny:
                nd = d + 0.5*(W[y0,x0] + W[y1,x1])  # midpoint rule cost
                if nd < dist[y1,x1]:
                    dist[y1,x1]=nd
                    prev[y1,x1]=[x0,y0]
                    heapq.heappush(pq, (nd, x1, y1))
    # reconstruct
    path=[]; cur=(gx,gy)
    if prev[gy,gx,0]==-1:
        return []
    while cur!=(sx,sy):
        path.append(cur)
        px,py = prev[cur[1],cur[0]]
        cur=(px,py)
    path.append((sx,sy))
    path.reverse()
    return path, dist[gy,gx]

start = (5, ny//2)
goal  = (nx-6, ny//2)
path, cost = dijkstra_path(W, start, goal)
len(path), cost

In [5]:
# Plot path bending over constraint mass
plt.figure(figsize=(6,5))
plt.imshow(W, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
if path:
    xs = [x[p[0]] for p in path]
    ys = [y[p[1]] for p in path]
    plt.plot(xs, ys, linewidth=2)
plt.title('Geodesic bending around a constraint cluster (higher traversal cost)')
plt.colorbar(label='local traversal cost ~ 1 + αΦ')
plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_Geodesic_bending.png', dpi=160)
plt.show()

## 4) Discussion & Bridge to GR
- **Constraint density** changes the effective metric on the state graph: edges in high-\(\Phi\) regions cost more and clocks tick slower → **time dilation** and **geodesic bending** emerge naturally.
- In the LFT picture, GR’s field equations should arise as **coarse-grained constraints** relating the distribution of \(\Phi\) (from matter/energy) to an effective metric on the emergent manifold.
- Next steps:
  1. Replace the lattice with **local permutohedral charts** from 02.
  2. Calibrate slowdown rule vs. **inversion-count dynamics** (03) to derive a specific functional form.
  3. Fit an **effective metric tensor** and test for analogs of redshift and lensing vs. GR predictions.
  4. Explore **constraint conservation laws** → analogs of stress-energy conservation.

### Artifacts
- `/mnt/data/LFT_12_Phi_field.png`
- `/mnt/data/LFT_12_Redshift_map.png`
- `/mnt/data/LFT_12_Geodesic_bending.png`